In [1]:
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor 

In [2]:
import os
embedding_function = OpenAIEmbeddings()

In [3]:
loader = TextLoader("/home/ankdesh/explore/LearnTry-ML/llm/Udemy-LangChain/01-Data-Connections/some_data/US_Constitution.txt")

In [4]:
doc = loader.load()

In [6]:
print (len(doc))

1


In [20]:
# split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=200)
docs = text_splitter.split_documents(doc)

Created a chunk of size 205, which is longer than the specified 200
Created a chunk of size 252, which is longer than the specified 200
Created a chunk of size 333, which is longer than the specified 200
Created a chunk of size 472, which is longer than the specified 200
Created a chunk of size 312, which is longer than the specified 200


In [21]:
print (len(docs))

89


In [22]:
db = Chroma.from_documents(docs, embedding_function)

In [23]:
model = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [24]:
compressor = LLMChainExtractor.from_llm(model)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=db.as_retriever())

In [25]:
compressed_docs = compression_retriever.get_relevant_documents("What is the 13th Amendment?")

/home/ankdesh/virtualenv/pytorch-host/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/ankdesh/virtualenv/pytorch-host/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/ankdesh/virtualenv/pytorch-host/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/ankdesh/virtualenv/pytorch-host/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [29]:
print (compressed_docs[1].page_content)

13th Amendment
Section 1
Neither slavery nor involuntary servitude, except as a punishment for crime whereof the party shall have been duly convicted, shall exist within the United States, or any place subject to their jurisdiction.


In [30]:
print ("\n".join(x.page_content for x in compressed_docs))

13th Amendment
13th Amendment
Section 1
Neither slavery nor involuntary servitude, except as a punishment for crime whereof the party shall have been duly convicted, shall exist within the United States, or any place subject to their jurisdiction.
